In [1]:
from openai import OpenAI, beta

In [2]:
openai_api_key = ""
OpenAI.api_key = openai_api_key

import os
os.environ["OPENAI_API_KEY"] = openai_api_key
client = OpenAI()

# Prompt

In [3]:
import pandas as pd

In [4]:
train_only = pd.read_csv("Data/train_v2_400.csv")
train_only.head()

,index,label,line
0,0,1,INV: this is the picture.\nPAR: mhm.\nINV: jus...
1,2,1,PAR: here's a cookie jar.\nPAR: and the lid is...
2,3,1,INV: this is the picture.\nINV: just tell me w...
3,5,1,INV: www .\nINV: there it is.\nPAR: xxx.\nINV:...
4,6,1,INV: on in the picture.\nPAR: well the girl is...


In [5]:
patient = train_only[train_only.label == 1]
control = train_only[train_only.label == 0]

# Batch production

In [6]:
from pydantic import BaseModel
import json

class Conversation_Type(BaseModel):
    conversation_1: str
    conversation_2: str
    conversation_3: str
    conversation_4: str
    conversation_5: str
    conversation_6: str
    conversation_7: str
    conversation_8: str
    conversation_9: str
    conversation_10: str

def generate_alzheimers_transcriptions(prompt, gpt_model):
    prompt = prompt

    # transcriptions = []
    response = beta.chat.completions.parse(
        model=gpt_model,
        messages=[
            {"role": "system", "content": "You are an Alzheimer’s patient"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.8,
        max_tokens= 8000,
        response_format= Conversation_Type
    )
    transcriptions = response.choices[0].message.content

    return transcriptions

In [7]:
def production(patient, gpt_model, loop):
  for i in range(0, len(patient["line"]), 10):
      lines_group = patient["line"][i:i+10]
      combined_text = "\n\n".join([f"Example:\n{line}" for line in lines_group])
      prompt_v3 = f"""
        Task:

        Generate ten unique and varied conversations of the scene as if you were the patient, incorporating language impairments common in Alzheimer’s disease.

        As an Alzheimer’s patient, your description should include features such as:
        • Reduced specificity: Use nonspecific terms like “thing,” “stuff.”
        • Difficulty recalling words: Pauses while searching for words.
        • Vague or incomplete descriptions of actions.
        • Semantic confusion: Mixing up words or meanings.
        • Use of fillers, pauses, and struggling speech sounds: Indicated by (.), (..), (...).
        • Misspelled words or single letters to represent sounds or hesitations: For example, “uh,” “um,” “er.”

        Instructions:
        • Do not use any other notations and markers beside "," "." "?"
        • Each description should start differently and use varied expressions.
        • Use pauses (.), (..), (...) to indicate short, medium, and long pauses, use them with the same frequency as in the examples, and do not over use them.
        • Avoid overusing any single phrase or sentence structure.
        • Incorporate language impairments naturally, reflecting the speech patterns of an Alzheimer’s patient.

        Please try to learn and generate conversations based directly on these shown examples, use the same format and same style, include both the conversation from investigator and the patient, you may start with 'INV: So here is the picture, just tell me everything that you see happening in that picture.'.

        {combined_text}
        """

      alzheimers_descriptions = generate_alzheimers_transcriptions(prompt_v3, gpt_model)
      alzheimers_descriptions_dict = json.loads(alzheimers_descriptions)

      alzheimers_descriptions_dict["example_index"] = patient['index'][i:i+10].to_list()

      # Save the transcriptions to a file
      with open(f"Results/transcriptions_{gpt_model}_v3_{i//10 + 1}_{loop}.json", "w") as file:
              json.dump(alzheimers_descriptions_dict, file, indent=4)
      print(f"Saved transcriptions for transcription number {i//10 + 1}")

In [9]:
# gpt_model="chatgpt-4o-latest"
gpt_model="gpt-4o-2024-08-06"   #Json parse version
for i in range(7, 10):
  print(f"Production loop {i}")
  production(patient, gpt_model, i)
  print()
print("Generation done!")

Production loop 7
Saved transcriptions for transcription number 1
Saved transcriptions for transcription number 2
Saved transcriptions for transcription number 3
Saved transcriptions for transcription number 4
Saved transcriptions for transcription number 5
Saved transcriptions for transcription number 6
Saved transcriptions for transcription number 7
Saved transcriptions for transcription number 8
Saved transcriptions for transcription number 9
Saved transcriptions for transcription number 10
Saved transcriptions for transcription number 11
Saved transcriptions for transcription number 12
Saved transcriptions for transcription number 13
Saved transcriptions for transcription number 14
Saved transcriptions for transcription number 15
Saved transcriptions for transcription number 16
Saved transcriptions for transcription number 17
Saved transcriptions for transcription number 18
Saved transcriptions for transcription number 19
Saved transcriptions for transcription number 20

Production

# Control group batch production

In [10]:
from pydantic import BaseModel

class Conversation_Type(BaseModel):
    conversation_1: str
    conversation_2: str
    conversation_3: str
    conversation_4: str
    conversation_5: str
    conversation_6: str
    conversation_7: str
    conversation_8: str
    conversation_9: str
    conversation_10: str

def generate_controls_transcriptions(prompt, gpt_model):
    prompt = prompt

    # transcriptions = []
    response = beta.chat.completions.parse(
        model=gpt_model,
        messages=[
            {"role": "system", "content": "You are an old person over 65 years old"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.8,
        max_tokens= 8000,
        response_format= Conversation_Type
    )
    transcriptions = response.choices[0].message.content

    return transcriptions

In [11]:
def control_production(control, gpt_model, loop):
  for i in range(0, len(control["line"]), 10):
      lines_group = control["line"][i:i+10]
      combined_text = "\n\n".join([f"Example:\n{line}" for line in lines_group])
      prompt_v3_control = f"""
      Generate ten unique and varied conversations of the cookie theft scene as if you were an old person over 65 years old.

      Instructions:
      • Do not use any other notations and markers beside "," "." "?"
      • Each conversation should start differently and use varied expressions.
      • Use pauses (.), (..), (...) to indicate short, medium, and long pauses, use them with the same frequency as in the examples, and do not overuse them.
      • Use Misspelled words or single letters to represent sounds, for example: “uh” “um” “er” (refer to the examples for more hesitation sounds)
      • Avoid overusing any single phrase or sentence structure.

      Please try to learn and generate conversations based directly on these shown examples, use the same format and same style, include both the conversation from investigator and the patient, you may start with 'INV: So here is the picture, just tell me everything that you see happening in that picture.'.

      {combined_text}
      """

      control_descriptions = generate_controls_transcriptions(prompt_v3_control, gpt_model)
      control_descriptions_dict = json.loads(control_descriptions)

      control_descriptions_dict["example_index"] = control['index'][i:i+10].to_list()

      # Save the transcriptions to a file
      with open(f"Results/control_transcriptions_{gpt_model}_v3_{i//10 + 1}_{loop}.json", "w") as file:
              json.dump(control_descriptions_dict, file, indent=4)
      print(f"Saved transcriptions for transcription number {i//10 + 1}")

In [12]:
# gpt_model="chatgpt-4o-latest"
gpt_model="gpt-4o-2024-08-06"   #Json parse version

for i in range(10):
  print(f"Production loop {i}")
  control_production(control, gpt_model,i)
  print()

print("Generation done!")

Production loop 0
Saved transcriptions for transcription number 1
Saved transcriptions for transcription number 2
Saved transcriptions for transcription number 3
Saved transcriptions for transcription number 4
Saved transcriptions for transcription number 5
Saved transcriptions for transcription number 6
Saved transcriptions for transcription number 7
Saved transcriptions for transcription number 8
Saved transcriptions for transcription number 9
Saved transcriptions for transcription number 10
Saved transcriptions for transcription number 11
Saved transcriptions for transcription number 12
Saved transcriptions for transcription number 13
Saved transcriptions for transcription number 14
Saved transcriptions for transcription number 15
Saved transcriptions for transcription number 16
Saved transcriptions for transcription number 17
Saved transcriptions for transcription number 18
Saved transcriptions for transcription number 19
Saved transcriptions for transcription number 20

Production